In [2]:
import serial

serial_port = "COM3"

def read_arduino(serial_port = "COM3", baudrate = 115200, timeout = 2):
    with serial.Serial(port = serial_port, baudrate = baudrate, timeout = timeout) as arduino:
        arduino.read_until(b'Yours sincerely, Arduino') # The end of our 'distinctive message', don't need to print anymore.
        print('\n') # Extra line for clarity
        value = arduino.read_until(b'\r\n') # remember that Serial.println() terminates with '\r\n', and it is a bytearray!
        try:
            numeric_value = float(value.decode().strip())
        except ValueError:
            numeric_value = 0.0
    return numeric_value

new_numeric_value = read_arduino()

print('new_numeric_value;', new_numeric_value, ', of type', type(new_numeric_value))



new_numeric_value; 1.64 , of type <class 'float'>


In [11]:
# .decode() changes from bytearray to string type
# .strip() removes extra lines/indentation.
# float() changes to type float

In [3]:
import numpy
import serial

def read_arduino(number_readings=10, data_type=numpy.float32, error_value=numpy.NaN,
                 serial_port='COM3', baud_rate=115200, timeout=2, 
                 terminator='\r\n', startup_message='Yours sincerely, Arduino'): 
    '''
    Returns number_readings (default 1) voltage measurements from Arduino attached to 
    serial_port at baud_rate with specified timeout.
    Each voltage reading from Arduino should be terminated with terminator (string), and 
    it should give a suitable startup message (string).  
    Returns either a numpy array containing a single value (num_readings <= 1), or of length 
    number_readings, of type data_type. 
    Returns element with error_value if value cannot be converted to type data_type.
    '''
    if number_readings < 1: number_readings = 1 # need at least 1 reading!
    # Define an empty array of appropriate size and datatype
    numeric_values = numpy.zeros((number_readings,), dtype=data_type)
    
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        # Connect, and read until distinctive startup_message
        arduino.read_until(startup_message.encode()) # string.encode() -> bytearray
        for i in range(number_readings): # Gather the appropriate number of measurements
            value = arduino.read_until(terminator.encode()) # string.encode() -> bytearray
            try: # try to convert returned value to type data_type
                numeric_values[i] = data_type(value.decode().strip())
            except ValueError: # If we fail to convert data to type data_type, then
                numeric_values[i] = error_value # return error_value instead.
    # Remember that exiting the 'with' block closes the connection automatically.
    return numeric_values

numpy_numeric_values = read_arduino()

print('numpy_numeric_values; ', numpy_numeric_values, 
      ', of type', type(numpy_numeric_values))
      
print('numpy_numeric_values[0]; ', numpy_numeric_values[0], 
      ', of type', type(numpy_numeric_values[0]))

mean = numpy.mean(numpy_numeric_values) 
std_error = numpy.std(numpy_numeric_values)/numpy.sqrt(len(numpy_numeric_values))

print('average value =', mean, '+/-', std_error, 'units')

numpy_numeric_values;  [1.64 1.64 1.64 1.64 1.64 1.64 1.64 1.64 1.64 1.64] , of type <class 'numpy.ndarray'>
numpy_numeric_values[0];  1.64 , of type <class 'numpy.float32'>
average value = 1.64 +/- 0.0 units


In [5]:
voltages = read_arduino(number_readings=10)
print(voltages)

[64.    1.64  1.64  1.63  1.64  1.64  1.64  1.63  1.64  1.64]
